<h1><center>Big Data Algorithms Techniques & Platforms</center></h1>
<h2>
<hr style=" border:none; height:3px;">
<center>Assignment 4 - MapReduce and Spark</center>
<hr style=" border:none; height:3px;">
</h2>

Official Assignment Submission By: <br>
Karim EL HAGE <br>
karim.elhage@student-cs.fr

# Introduction


<p align="justify">
<font size="3">
In this exercise you is asked to use Spark for implementing an algorithm that applies computations on documents and dataframes.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize Spark**</font>
<hr style=" border:none; height:2px;">
</p>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar zxvf spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark
 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"


import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local")
sc = SparkContext(conf = conf)
print("initialization successful!")


#import of the SparkSession
from pyspark.sql import SparkSession


import findspark
findspark.init()

#import of the SparkSession
from pyspark.sql import SparkSession

#inizialization of the Spark Session
spark = SparkSession \
    .builder \
    .appName("Assignment_4") \
    .getOrCreate()

import numpy as np
import random as rn
import pandas as pd

seed_value=0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# Analysing documents


<p align="justify">
<font size="3">
We have already seen that MapReduce procedures are good in analyzing text-files.
    
The provided data comes from The Office serie and is available on kaggle:
    
https://www.kaggle.com/datasets/thedevastator/the-office-character-dialogues
    

    
The assignment is divided in 3 parts:
    
* Part 1 is focused on MapReduce;
    
* Part 2  is focuses on dataframes;

* Part 3 is on MongoDB (another file).

    </font>
    </p>
    
<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>Notice that if you see that your colab instance is not supporting all the documents you can work on a subset of them.</font>
<hr style=" border:none; height:2px;">
</p>

## Library Import

In [2]:
#Libraries to import
import re 
import string
import nltk

#Download English Stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_ENG = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from pyspark.sql.window import Window #Function to help map enumeration to each value within specific key
from pyspark.sql.functions import rtrim,col,isnan, when, count, split,posexplode,row_number,size, round, monotonically_increasing_id ,lower,regexp_replace,array_join,ltrim

#rtrim - remove trailing blank space
#ltrum - remove leading blank space
#col - to call a column
#isnan - to check if nan value
#when - conditional criteria
#count - counts 
#split - splts string within column based on specific criteria
#posexplode - explodes a list in a column such that value in the list creates a new row with all the other respective related columns. Also assigns position in list to new column
#row_number - finds row number
#size - returns of length of item within column
#round - rounds value
#monotonically_increasing_id - Assigns an index to each row in dataframe in a new column
#lower - lower cases string
#regexp_replace - allows replacement of string
#array_join - allows joining of array

## Data Import and Cleaning

Prior to handling the assignment, it is necessary to preprocess the data due to several typos, errors, and inconsistencies found during data assessment.

In [5]:
#There are several episodes that do not constistute as official episodes of the series such as deleted scenes or unofficial episodes.
#These must be excluded from analysis. Since "the_office_series.csv" contains the official list of episodes, it shall be used to filter
#out episodes not pertinent to the analysis. The task proved to be slightly challenging due the variation in names between the episode names in 
#this csv files and those in the trancripts csv as well as txt file. In hindsight, tt might have been more efficient to employ either to employ a fucntion that looks
#for similarity in title names rather exact matches or to filter using the episode number (although this solution is also not perfect as it some manual pre-processing 
#is required since some episodes in two parts are labelled as for eg Ep 02x22/23 rather than 02x23 and 02/24 seperately). It could be argued to seperate such episodes
#using a split but since the dialogue is presented as one, we would have to infer where each episodes ends which could be a guessing task as there does not
#seem to be a cutting point such as "end of first episode. Another method could be to just split the file in half as an approxmation for each of the two episodes.
#For the purpose of this assignment, the filter was done by exact matches of episode names rather than numbers and any deviations were manually rectified.

#The multi line option has been used to deal with the fact that in some lines there are odd symbols in some of the fields. We will also skip over , within "". If this is not done
#The csv file will not be read correctly and will give the impression as though some fields have null values when they in fact fo not
df_office = spark.read.options(header='True',inferSchema=True,sep=',',escape = '"',multiLine = True).csv("/Dataset/the_office_series.csv")
df_office.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- EpisodeTitle: string (nullable = true)
 |-- About: string (nullable = true)
 |-- Ratings: double (nullable = true)
 |-- Votes: integer (nullable = true)
 |-- Viewership: double (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- GuestStars: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writers: string (nullable = true)



In [6]:
#We will first clean the office transcripts file to only have relevant episodes.
df_trans = spark.read.options(header='True',inferSchema=True,delimiter=';').csv("/Dataset/TheOffice_transcripts.csv")
df_trans.printSchema()

root
 |-- Field: string (nullable = true)
 |-- Field1: string (nullable = true)
 |-- Text: string (nullable = true)



In [7]:
#Renaming of fields
df_trans = df_trans.select(col("Text").alias("Title"),col("Field1").alias("Character"),col("Field").alias("Dialogue"))
df_trans.show()

+-------------+---------+--------------------+
|        Title|Character|            Dialogue|
+-------------+---------+--------------------+
|01x01 - Pilot|  Michael|Michael: All righ...|
|01x01 - Pilot|      Jim|Jim: Oh, I told y...|
|01x01 - Pilot|  Michael|Michael: So you'v...|
|01x01 - Pilot|      Jim|Jim: Actually, yo...|
|01x01 - Pilot|  Michael|Michael: All righ...|
|01x01 - Pilot|  Michael|Michael: [on the ...|
|01x01 - Pilot|  Michael|Michael: I've, uh...|
|01x01 - Pilot|      Pam|Pam: Well. I don'...|
|01x01 - Pilot|  Michael|Michael: If you t...|
|01x01 - Pilot|      Pam|          Pam: What?|
|01x01 - Pilot|  Michael|Michael: Any mess...|
|01x01 - Pilot|      Pam|Pam: Uh, yeah. Ju...|
|01x01 - Pilot|  Michael|Michael: Oh! Pam,...|
|01x01 - Pilot|      Pam|Pam: You haven't ...|
|01x01 - Pilot|  Michael|Michael: It's cal...|
|01x01 - Pilot|  Michael|"Michael: People ...|
|01x01 - Pilot|   Dwight|Dwight: [singing]...|
|01x01 - Pilot|      Jim|Jim: My job is to...|
|01x01 - Pilo

In [8]:
#In the "Title" field corresponding, the missing values
#can be inferred easily since the data is ordered. The missing value can be inffered by looking at the rows immeidately above/below the missing value. Ofcourse,
#if the above an dbelow value are not the same, a manual check would have to to be done to infer the correct position of the dialogue.

df_trans.select([count(when(isnan(column) | col(column).isNull(), column)).alias(column) for column in df_trans.columns])\
                .show()

+-----+---------+--------+
|Title|Character|Dialogue|
+-----+---------+--------+
|    4|     1167|       0|
+-----+---------+--------+



In [9]:
# Null titles are filled with the title names in the dialogue directly above/below them. This is done by inserting an index to each row to be able
#to track the locationof the null values. We the then extract the ids where we have null titles in a list. Then for each id for a null title we replace 
#the title with the name of the title of the title directly preceeding it. 
df_trans = df_trans.withColumn("id",monotonically_increasing_id())
null_titles = df_trans.where(col("Title").isNull()).select("id").rdd.map(lambda x: x[0]).collect()

for idx in null_titles:
 df_trans =  df_trans.withColumn("Title",when(col("Title").isNull(),\
                                   df_trans.where(df_trans.id == idx-1)\
                                   .select("Title").rdd.map(lambda x: x[0]).collect()[0])\
                    .otherwise(col("Title")))

In [ ]:
#The majority "Character" missing values can be ignored from analysis as they correspond to an unreleased episode 09x26 that will be filtered out
#once we filter out the rows to only contain the official episodes list.
#The remaining null values can be filled using the person speaking in "Dialogue" column as in each row of dialogue we have "Person Speaking:". And so 
#we take the string existing before the ":" symbol. Any null values left after this
#corrrespond to actions by characters that are not dialogue  such as when a person exits a room or enters a room. 
#These lines will be filtered out from analysis.
df_trans.select([count(when(isnan(column) | col(column).isNull(), column)).alias(column) for column in df_trans.columns])\
                .show()

+-----+---------+--------+---+
|Title|Character|Dialogue| id|
+-----+---------+--------+---+
|    0|     1167|       0|  0|
+-----+---------+--------+---+



In [12]:
episodes_list=df_office.rdd.map(lambda x: x[2]).collect() #Extracting official episode list from "The_office_series.csv"

#In case there are any wrong titles included such as pure number or punctuation, these will be filtered out first by taking the 
#non text length of each line and ensuring there is still text that exists.
episodes_list = list(dict.fromkeys([title for title in episodes_list if len(re.sub('[^a-zA-Z ]',"",title))>0])) 

In [14]:
#An edited episodes list is created so that we remove the part numbers from episode names to compare matches
#only with the name of the episode. This is to help matching the episode title names in the "the office series.csv" compared to the 
#transcripts file due to variations in how the titles are written. In hindisght it would have been easier to compare against the numerical episode tital
#rather than the name of the episode.
#The episode name A.A.R.M was also modified as matching it provied to be challenging
edited_list = [episode.split(":")[0].lower() for episode in episodes_list]
edited_list[-2] = "aarm"

In [18]:
#Now that we have our official episodes list from the office series, we move to the transcripts csv file to filter out any non offial episodes.
#We do this by first creating a new column that contains of the text name of the episode. This is done by parisong for the "-" symbol of the episode title
#and taking whatecer exists to the right. 

#All punctuation removed and string lowered for easier  matching. we also use a trim to remove any to remove trailing white blanks. For 9 episodes, manual 
#matching needed due to text incosistinces between how the name of the episode is written in both files. Also, the part numbers
#are removed (by removing text in brackets) for better matching (if episodes have a part 1 & part 2). We then filter to have a data frame with only the
#official episode title list. I again reiterate how I wished in hindsight to instead compare ahaionst the numerical title rather than text title of the 
#episode.
df_trans = df_trans.withColumn("EpisodeTitle",split(col("Title")," - ")[1])\
.withColumn("EpisodeTitle",ltrim(rtrim(regexp_replace(col("EpisodeTitle"),"[\(\[].*?[\)\]]",""))))\
.withColumn("EpisodeTitle",when(col("Title").startswith("02x15"),"Boys and Girls").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",when(col("Title").startswith("05x18"),"Blood Drive").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",when(col("Title").startswith("05x23"),"Michael Scott Paper Company").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",when(col("Title").startswith("06x10"),"Murder").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",when(col("Title").startswith("06x16"),"Manager and Salesman").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",when(col("Title").startswith("07x24"),"Dwight K. Schrute, (Acting) Manager").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",when(col("Title").startswith("08x23"),"Turf War").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",when(col("Title").startswith("09x01"),"New Guys").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",when(col("Title").startswith("09x22"),"aarm").otherwise(col("EpisodeTitle")))\
.withColumn("EpisodeTitle",lower(col("EpisodeTitle")))\
.filter(col("EpisodeTitle").isin(edited_list))

In [19]:
#The rest of the cleaning will be done in part 2 of the excercises
df_trans.select([count(when(isnan(column) | col(column).isNull(), column)).alias(column) for column in df_trans.columns])\
                .show()

+-----+---------+--------+---+------------+
|Title|Character|Dialogue| id|EpisodeTitle|
+-----+---------+--------+---+------------+
|    0|      220|       0|  0|           0|
+-----+---------+--------+---+------------+



In [ ]:
#We now analyze the txt transcript file to be used in Problem 1
!unzip "/Dataset/the_office.zip"

In [23]:
#Using the wholeTextFiles function, we can read the txt files in a manner such that each file is mapped to an entry. Each entry of the RDD is a list of 2 entries: the name of the file
#and the contents of the file. We only need the second. 
#We notice that each line in the contents is sperated with a \n. so we can now parse the seperate lines using the "\n" as seperater.
#This will give us 3-4 parses: The Title, Character, Dialogue, and sometimes a blank space. So we only take the first 3 from the split

rdd = sc.wholeTextFiles("the_office/*").map(lambda line: line[1].split("\n")[0:3])

In [24]:
#Since the Episode Titles in the transcripts csv file are an match to those in the txt files, we shall use an updated episodes
#list from the the now cleaned transcripts csv file and use it to filter out any non official episodes from the txt transcripts file.

episodes_list=df_trans.select("Title").distinct().rdd.map(lambda x: x[0]).collect()
rdd = rdd.filter(lambda line: line[0] in episodes_list)

## Helper Functions

In [25]:
def clean_dialogue(line): #Helper function to clean dialogue and remove stop words. This will be used to find common words
  stopwords_ENG = stopwords.words('english')

  line = re.sub("[\(\[].*?[\)\]]", "", line) #removes text between sqaure brackets since these are character actions rather than dialogue
  line = line[line.find(':'):] #remove texts before ":" since this does not correspond to dialogue and is the name of the person speaking"
  line = re.sub('[^a-zA-Z ]',"",line) #removes all non letter characters
  line = [word.lower() for word in line.split(" ") if (len(word)>0) & (word.lower() not in stopwords_ENG)] #Each lines is then split into words
  #by parsing blank space. any word that is a stop word is remove and then lower case for better gouping.

  return line # the output is list of words due to the parsing operation


In [26]:
def clean_dialogue_sw(line): #Helper function that cleans dialogue but keeps stop words. Rest of explanation is same as above
  stopwords_ENG = stopwords.words('english')

  line = re.sub("[\(\[].*?[\)\]]", "", line)
  line = line[line.find(':'):]
  line = re.sub('[^a-zA-Z ]',"",line)
  line = [word.lower() for word in line.split(" ") if (len(word)>0)] #dialogue is split where spaces exists so output


  return line #list of words


In [27]:
def clean_character(char_name): #takes as an input the character's name as a string
  char_name = re.sub("[\(\[].*?[\)\]]", "", char_name) #removes any words that are inside of brackets since it refers to action or not the character's name
  #such as ("Pam's Mom") or [over the phone]
  char_name = re.sub('[^a-zA-Z ]',"",char_name) #removes any punctuation from character's name
  char_name = char_name.strip().lower() #removes any trailing and leading white spaces from the name and lower cases the name

  return char_name #the cleaned character name as a string

# Part 1 -  MapReduce
<p align="justify">
<font size="3">
In the provided folder you can find a folder called <code>the_office</code>. The folder contains set (<code>59582</code>) of documents/files containing  the dialogs (words and additional informations referring to the dialog). 
Specifically in each file:

- the first line is the episode ID;
- the second line is the name of the character that is speaking;
- from the second line is the content of one dialog.
    </font>
    </p>



## Exercise 1 - (2 points) - Dialog number for episode

<p align="justify">
<font size="3">
Using just the informations provided in the files provide a map-reduce procedure that counts how many dialogs are present for each episode.

 </font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>Notice that  you can reuse any code that was made available for the previous labs/assignments or that you already developed in these contexts.</font>
<hr style=" border:none; height:2px;">
</p>

In [28]:
#A simple Map Reduce by taking the episode title as key and counting the number of times the title is repeating
#since the number of titles is repeated for the number of dialogues. However, we need to be wary that not all lines correspond to a dialogue as they could be an action
#such as [Michael walks to the office]. These lines do not have a character in the second field. It is not completely correct to solve this by filtering out locations
#that have a character value of '' as there are in fact also some dialogues that have a missing character due to error. Therefore, the most correct way would be to filter
#out locations that do not have ":" symbol in dialogue as in every location there is a dialogue, we have the character name followed by ":" before the dialogue begins.
#To do this, we will find the index of the location of ":". If the locations happens to be -1, it means there is no dialogue since either no dialogue is following the ":" symbol 
#or the ":" does not exist to begin with in the line.

rdd.filter(lambda line: line[2].find(":") != -1).map(lambda line: (line[0],1)).reduceByKey(lambda x, y: x+y).sortByKey().collect()

[('01x01 - Pilot', 224),
 ('01x02 - Diversity Day', 200),
 ('01x03 - Health Care', 239),
 ('01x04 - The Alliance', 241),
 ('01x05 - Basketball', 227),
 ('01x06 - Hot Girl', 342),
 ('02x01 - The Dundies', 252),
 ('02x02 - Sexual Harassment', 278),
 ('02x03 - Office Olympics', 279),
 ('02x04 - The Fire', 310),
 ('02x05 - Halloween', 262),
 ('02x06 - The Fight', 265),
 ('02x07 - The Client', 248),
 ('02x08 - Performance Review', 265),
 ('02x09 - E-Mail Surveillance', 270),
 ('02x10 - Christmas Party', 278),
 ('02x11 - Booze Cruise', 219),
 ('02x12 - The Injury', 352),
 ('02x13 - The Secret', 258),
 ('02x14 - The Carpet', 253),
 ('02x15 - Boys & Girls', 276),
 ("02x16 - Valentine's Day", 195),
 ("02x17 - Dwight's Speech", 201),
 ('02x18 - Take Your Daughter to Work Day', 262),
 ("02x19 - Michael's Birthday", 245),
 ('02x20 - Drug Testing', 211),
 ('02x21 - Conflict Resolution', 251),
 ('02x22 - Casino Night', 356),
 ('03x01 - Gay Witch Hunt', 139),
 ('03x02 - The Convention', 290),
 ('03x0

## Exercise 2 - (3 total points)
The most common words



### 2.1 - (1 point) - Common words by serie
Provide a Spark MapReduce procedure that returns the 10 most common words in the whole set of dialogs.




In [29]:
#First we filter out lines that completely do not have dialogue. Then, dialogue is first cleaned using helper function defined "clean dialogue" which removes words inside of 
#[] since these are actions rather than dialogue, then removes any non letter words, remoces stop words, and then the words are split into a list and lowercases (any non word is removed from this list) 
#.The results a flattened with a flatmap so that each word is now on a seperater line. each word is now mapped against a count and reduced. The results
#are sorted by the value that corresponds to the count and we take 10 entries to get the 10 mos common words.

rdd.filter(lambda line: line[2].find(":") != -1).map(lambda line: clean_dialogue(line[2])).flatMap(lambda line: line)\
    .map(lambda word: (word,1))\
    .reduceByKey(lambda x, y: x+y).sortBy(lambda f: f[1], ascending=False).take(10)

[('im', 4321),
 ('know', 3909),
 ('oh', 3859),
 ('dont', 3619),
 ('like', 2953),
 ('yeah', 2773),
 ('okay', 2528),
 ('well', 2496),
 ('right', 2381),
 ('get', 2318)]

### 2.2 - (1 point) - Common words by episode
Provide a Spark MapReduce procedure that returns the 10 most common words for each episode.




In [31]:
#First we filter out lines that completely do not have dialogue.The dialogue (cleaned as in 2.1) and title are taken are taken as key. Now having 
#the episode title and a list of words, we use a flat map to to turns the list of words into a line for each word but also leverage string minipulation
#to map the episode title to each word. Each of these pair keys (title, word) are assigned a count and reduced and sorted by value. Since we want
# the top 10 per episode, we then enumerate, over the ordered count per where the enumeration restarts at each episode episode (the episode title is the key and each word with its count is the value).
#we filter for all values per key that have an enumeration <=10 (<10 since enumeration starts at 0) to return the 10 most common per episode. We then change
#the mapping so that now the episode title is the key and the value being the word with its count. We group by key so that for each episode, we have
#a list of all the words contained with their respective count. since we already sorted by count before, we now enumerate an index to each value pair
#per key (episode title). Now we have have a value that contained the trio word, count, index. We now take only the words per key that have an index 
#less than 10 (since starting at 0). We sort by key (episode title) and have a list of words by epeisode title that are the top 10 common words
#per episode

rdd.filter(lambda line: line[2].find(":") != -1).map(lambda line: (clean_dialogue(line[2]),line[0]))\
    .flatMap(lambda line: [(line[1],word) for word in line[0]]).map(lambda word: (word, 1))\
                          .reduceByKey(lambda x,y: x+y)\
                          .sortBy(lambda f:f[1], ascending = False)\
                          .filter(lambda word: len(word[0][0])>0).map(lambda word: (word[0][0],(word[0][1],word[1])))\
                          .groupByKey().map(lambda line: (line[0],[(count[1],count[0]) for count in enumerate(line[1])]))\
                          .map(lambda line: (line[0], [count[0] for count in line[1] if count[1]<10])).sortByKey().collect()

[('01x01 - Pilot',
  [('yeah', 28),
   ('im', 23),
   ('ok', 21),
   ('know', 20),
   ('right', 18),
   ('pam', 17),
   ('going', 17),
   ('dont', 16),
   ('think', 14),
   ('thats', 14)]),
 ('01x02 - Diversity Day',
  [('ok', 34),
   ('im', 27),
   ('oh', 18),
   ('dont', 18),
   ('going', 17),
   ('right', 17),
   ('well', 17),
   ('know', 17),
   ('lets', 16),
   ('thank', 15)]),
 ('01x03 - Health Care',
  [('im', 26),
   ('know', 24),
   ('ok', 24),
   ('oh', 23),
   ('dont', 21),
   ('plan', 17),
   ('go', 16),
   ('well', 15),
   ('um', 15),
   ('dwight', 14)]),
 ('01x04 - The Alliance',
  [('know', 32),
   ('good', 26),
   ('dont', 25),
   ('oh', 23),
   ('um', 20),
   ('meredith', 20),
   ('ok', 19),
   ('think', 18),
   ('get', 17),
   ('uh', 17)]),
 ('01x05 - Basketball',
  [('know', 29),
   ('im', 28),
   ('right', 26),
   ('yeah', 26),
   ('come', 19),
   ('oh', 18),
   ('uh', 17),
   ('ok', 15),
   ('team', 15),
   ('like', 14)]),
 ('01x06 - Hot Girl',
  [('im', 33),
   ('

### 2.3 - (1 point) - Common words by character
Provide a Spark MapReduce procedure that returns the 10 most common words for each character.



In [33]:
##First we filter out lines that completely do not have dialogue as done in previous steps. 
#As discussed previously, there are some lines with no values in the character name field but do in fact have dialogue. 
#Therefore, the character name, shall be extracted from the dialogue by the string that exist before the ":" sign in the dialogue field
#and use this as our character. In addition, sometimes we have more than one character related to the same dialogue such as "Andy & Pam: Andy,Creed,Toby: etc"
#indicated that one line fo dialogue can be assocuated to several characters. so we extract these character names by parsing into a list of characters
#whenever exists ", and & /" as these are the punctuatons that split the diffent character. We use a flat map here so that each character in these lists 
# is mapped to the dialogue. The character names are cleaned using the "clean_character" helper function to ensure to remove any words between Brackets 
#such Michael [on the Phone] or Beth (Pam's Mom), remove punctuation, lower case, and remove any leading/trailing/spaces. THis solves most of the issues 
#to ensure we have the most distinct keys (character name) as possible. However, there are typos in some of the names such as Michael Michal. 
#In practice, it might be useful to employ some a function that makes use of Jaccard Similarity or min hashing to attempt to find similar
#character names and consider these as the same key if very similar. The key is first taken as this new character name and dialogue
#(cleaned as per previous steps). The rest is the same procedure as 2.2

rdd.filter(lambda line: line[2].find(":") != -1)\
.flatMap(lambda line: [(individual,line[2]) for individual in re.split(", |and |& |/",line[2][:line[2].find(":")])])\
.map(lambda line: (clean_character(line[0]),line[1]))\
.map(lambda line: (line[0],clean_dialogue(line[1])))\
.filter(lambda line: len(line[1])>0).flatMap(lambda line: [(line[0],word) for word in line[1]]).map(lambda word: (word, 1))\
                          .reduceByKey(lambda x,y: x+y)\
                          .sortBy(lambda f:f[1], ascending = False)\
                          .filter(lambda word: len(word[0][0])>0).map(lambda word: (word[0][0],(word[0][1],word[1])))\
                          .groupByKey().map(lambda line: (line[0],[(count[1],count[0]) for count in enumerate(line[1])]))\
                          .map(lambda line: (line[0], [count[0] for count in line[1] if count[1]<10])).sortByKey().collect()

[('aaron rodgers',
  [('youre', 3),
   ('man', 2),
   ('lot', 2),
   ('good', 2),
   ('middleaged', 1),
   ('makeup', 1),
   ('flag', 1),
   ('play', 1),
   ('look', 1),
   ('terrible', 1)]),
 ('abby',
  [('yeah', 2),
   ('mmhmm', 1),
   ('mixedup', 1),
   ('files', 1),
   ('mrs', 1),
   ('basil', 1),
   ('e', 1),
   ('frankweiler', 1),
   ('thanks', 1),
   ('thats', 1)]),
 ('abe',
  [('erin', 1),
   ('respect', 1),
   ('privacy', 1),
   ('follow', 1),
   ('everytime', 1),
   ('go', 1),
   ('thats', 1),
   ('takes', 1)]),
 ('actor',
  [('hey', 2),
   ('settle', 1),
   ('bet', 1),
   ('natural', 1),
   ('redhead', 1),
   ('rach', 1),
   ('im', 1),
   ('even', 1),
   ('scene', 1)]),
 ('ad guy',
  [('really', 3),
   ('okay', 3),
   ('hey', 3),
   ('sounds', 2),
   ('lets', 2),
   ('michael', 2),
   ('well', 2),
   ('dont', 2),
   ('ambitious', 1),
   ('im', 1)]),
 ('aj',
  [('im', 5),
   ('great', 3),
   ('hey', 3),
   ('michael', 3),
   ('see', 3),
   ('kind', 2),
   ('guys', 2),
   ('li

# Exercise 3 (3 total points)








## 3.1 - (1 point) - Character/Dialog 
Provide a Spark MapReduce procedure that returns the character that has the most number of dialogs (with the number of dialogs).

In [ ]:
#Non dialogue lines filtered out as before. Since we only need to count number of dialogue, the key we need is only the character name. 
#The character name is extracted as 2.3 to resolve the issues discussed above. Each cleaned character name is assigned a count of 1 at each occurence.
#Reduce and sort by highest value. Take highest value

rdd.filter(lambda line: line[2].find(":") != -1).flatMap(lambda line: [individual for individual in re.split(", |and |& |\\/",line[2][:line[2].find(":")])])\
.map(lambda char_name: (clean_character(char_name),1)).reduceByKey(lambda x,y: x+y).sortBy(lambda f: f[1], ascending=False).take(1)

[('michael', 10698)]

## 3.2 - (1 point) - Episode/Dialogs
Provide a Spark MapReduce procedure that returns the episode that has the most part of dialogs.





In [ ]:
#Filter out non-dialogue lines and take episode title as key as w ehave as many epieosde title as dialogue. Map a value of one to each occurence.
#redude, sort by value and take the first one.

rdd.filter(lambda line: line[2].find(":") != -1).map(lambda line: (line[0],1)).reduceByKey(lambda x,y: x+y).sortBy(lambda f: f[1], ascending=False).take(1)

[('06x17/18 - The Delivery (Parts 1&2)', 597)]

## 3.3 - (1 point) - Words/Episode
Provide a Spark MapReduce procedure that returns the 10 episodes having the most number of words.

In [ ]:
#First we filter out non-dialogue becaause, although these are words, they are not "spoken" words and so the viewer does not actually experience
#them. Since the count of words is investigated, stop words should not be removed and so the second helper function clean dialogue sw is used. 
#The rest of the dialogue cleaning procedure is exactly the same as described before. Each episode is taken as key and the number of words per line of 
#dialogue as value. Since, the output of clean_dialogue_sw is a list of words, we merely need to take the length of each list to find the number of 
#words per line of dialogue. we then reduce that combine these different list lengths per episode title (as key), we sort by value and take the 
#top 10. 

rdd.filter(lambda line: line[2].find(":") != -1).map(lambda line: (line[0],len(clean_dialogue_sw(line[2])))).reduceByKey(lambda x,y: x+y)\
.sortBy(lambda f: f[1], ascending=False).take(10)

[('07x25/26 - Search Committee (Parts 1&2)', 6082),
 ('06x17/18 - The Delivery (Parts 1&2)', 6008),
 ('09x24/25 - Finale', 5950),
 ('07x11/12 - Classy Christmas (Parts 1&2)', 5939),
 ('06x04/05 - Niagara (Parts 1&2)', 5555),
 ('04x03/04 - Dunder Mifflin Infinity (Parts 1&2)', 5448),
 ('04x05/06 - Launch Party (Parts 1&2)', 5376),
 ('04x01/02 - Fun Run (Parts 1&2)', 5301),
 ('09x16 - Moving On', 5286),
 ('05x01/02 - Weight Loss (Parts 1&2)', 5249)]


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**DataFrames**</font>
<hr style=" border:none; height:2px;">
</p>

# Part 2 - Dataframes

In this part you can use Pandas Dataframes or  Spark Dataframes.  I suggest to use a Spark Dataframe
end exploit the Pandas functionalities as we have seen in the 2nd assignment. Download the 3 available datasets at the link:

https://www.kaggle.com/datasets/thedevastator/the-office-character-dialogues

You can find 3 .cvs files: 

* the_office_series.csv

* TheOffice_data.csv

* TheOffice_transcripts

We will focus on the file2:

* the_office_series.csv

* TheOffice_transcripts.csv



# Import  data.
<p align="justify">
<font size="3">

The <code> the_office_series </code> file contains 11 fields. Not all the fields are of interest for the analysis. 

* Season	
* EpisodeTitle	
* About	
* Ratings	
* Votes	
* Viewership	
* Duration	
* Date	
* GuestStars	
* Director	
* Writers
    
</font>
</p>




In [34]:
df_office.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- EpisodeTitle: string (nullable = true)
 |-- About: string (nullable = true)
 |-- Ratings: double (nullable = true)
 |-- Votes: integer (nullable = true)
 |-- Viewership: double (nullable = true)
 |-- Duration: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- GuestStars: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writers: string (nullable = true)



In [35]:
#The null values are located in column  that is nor relevant to the excercises but, if needed, can be filled witht he value "None"
#since it is possible to not have Guest Stars on the show. This was checked by checking for each column where there are null or nan values.

df_office.select([count(when(isnan(column) | col(column).isNull(), column)).alias(column) for column in df_office.columns])\
                .show()

+---+------+------------+-----+-------+-----+----------+--------+----+----------+--------+-------+
|_c0|Season|EpisodeTitle|About|Ratings|Votes|Viewership|Duration|Date|GuestStars|Director|Writers|
+---+------+------------+-----+-------+-----+----------+--------+----+----------+--------+-------+
|  0|     0|           0|    0|      0|    0|         0|       0|   0|       159|       0|      0|
+---+------+------------+-----+-------+-----+----------+--------+----+----------+--------+-------+



#  Exercise 4 - (1 point) - Director list

<p align="justify">
<font size="3">
Provide a program that finds the distinct list of the directors.

</font>
</p>






In [ ]:
#We take the Director Column and first manipulate the column in order to not have duplicates due to trailing/leading spaces, punctuations and upper/lowercase mismatches. We then 
#take the distinct number of entries

df_office.select(col('Director')).withColumn("Director",regexp_replace(lower(rtrim(ltrim(col("Director")))),'[^a-zA-Z ]',"")).distinct().show(200, truncate = False)

+--------------------+
|Director            |
+--------------------+
|jennifer celotta    |
|jesse peretz        |
|lee eisenberg       |
|john scott          |
|jon favreau         |
|miguel arteta       |
|craig zisk          |
|tucker gates        |
|joss whedon         |
|brent forrester     |
|john krasinski      |
|gene stupnitsky     |
|paul lieberstein    |
|victor nelli jr     |
|julian farino       |
|lee kirk            |
|dennie gordon       |
|matt sohn           |
|ed helms            |
|ken kwapis          |
|mindy kaling        |
|bryan gordon        |
|david rogers        |
|claire scanlon      |
|daniel chun         |
|marc webb           |
|ken whittingham     |
|asaad kelada        |
|jason reitman       |
|brian baumgartner   |
|kelly cantleykashima|
|steve carell        |
|roger nygard        |
|stephen merchant    |
|paul feig           |
|harold ramis        |
|dean holland        |
|reginald hudlin     |
|eric appel          |
|charles mcdougall   |
|bj novak  

#  Exercise 5 - (2 points) - Director/Episodes

<p align="justify">
<font size="3">
Provide the code that provides how many episodes each director directed.

</font>
</p>




In [ ]:
#Director is first cleaned as descirbed in Excercise 4, we then take the director column and perform a group by  and count the 
#occurences and can sort by the highest count

df_office.withColumn("Director",regexp_replace(lower(rtrim(ltrim(col("Director")))),'[^a-zA-Z ]',""))\
.select(col('Director'),col('EpisodeTitle')).groupBy('Director').count().orderBy(col("count").desc()).show(100)

+--------------------+-----+
|            Director|count|
+--------------------+-----+
|     randall einhorn|   15|
|           paul feig|   15|
|          ken kwapis|   13|
|        greg daniels|   13|
|       jeffrey blitz|   11|
|        david rogers|    9|
|   charles mcdougall|    8|
|           matt sohn|    8|
|     ken whittingham|    8|
|    see full summary|    7|
|    paul lieberstein|    7|
|            bj novak|    5|
|        harold ramis|    4|
|        tucker gates|    4|
|     brent forrester|    4|
|        steve carell|    3|
|         troy miller|    3|
|        rainn wilson|    3|
|    jennifer celotta|    3|
|      john krasinski|    3|
|         seth gordon|    2|
|         joss whedon|    2|
|            ed helms|    2|
|        bryan gordon|    2|
|        mindy kaling|    2|
|       julian farino|    2|
|      claire scanlon|    2|
|        dean holland|    2|
|        jesse peretz|    1|
|         jon favreau|    1|
|       lee eisenberg|    1|
|       miguel

#  Exercise 6 - (2 points) Writer/Episode

<p align="justify">
<font size="3">
Provide the code that using dataframe return the number of episodes each writer contributed to.
</font>
</p>

In [ ]:
#Since an episode can have more than one writer, the cell first needs to be split and exploded so that each writer receives a contribution for episode.
#Writer names are either sperated by "and" or "|" and so these will be used to perform parse. The output is position exploded and we have the name of
#each writer and the position in the text they were in relative to the split (before or after the spliting criteria). We are not insterested in the position 
#so only take the name of the writers. The name of the writers is cleaned as in Excercise 5 to remove any mismatches and then we can group by the writers 
#and order by the highest count

df_office.select(posexplode(split("Writers", " and |\\|")).alias("pos", "Writers")).select("Writers")\
.withColumn("Writers",regexp_replace(lower(rtrim(ltrim(col("Writers")))),'[^a-zA-Z ]',""))\
.groupBy('Writers').count().orderBy(col("count").desc()).show(100, truncate = False)



+-------------------+-----+
|Writers            |count|
+-------------------+-----+
|mindy kaling       |22   |
|paul lieberstein   |16   |
|lee eisenberg      |15   |
|gene stupnitsky    |15   |
|b j novak          |15   |
|greg daniels       |13   |
|jennifer celotta   |11   |
|brent forrester    |11   |
|justin spitzer     |11   |
|michael schur      |10   |
|halsted sullivan   |7    |
|warren lieberstein |7    |
|daniel chun        |7    |
|charlie grandy     |7    |
|aaron shure        |6    |
|carrie kemper      |4    |
|robert padnick     |4    |
|owen ellickson     |4    |
|steve hely         |3    |
|allison silverman  |3    |
|dan sterling       |2    |
|ryan koh           |2    |
|amelie gillette    |2    |
|dan greaney        |2    |
|graham wagner      |2    |
|jon vitti          |2    |
|stephen merchant   |2    |
|gabe miller        |2    |
|niki schwartzwright|2    |
|jonathan green     |2    |
|steve carell       |2    |
|ricky gervais      |2    |
|anthony q farrell  

# Import  data.
<p align="justify">
<font size="3">

The <code> TheOffice_transcripts </code> file contains 3 columns.  

* Field	: contains the dialog
* Field1	: contains the name of the speaking character
* Text : contains the Episode ID
    
</font>
</p>




In [ ]:
#We now continue the cleaning of the transcript file that was done at the beginning of the note book
df_trans.select([count(when(isnan(column) | col(column).isNull(), column)).alias(column) for column in df_trans.columns])\
                .show()

+-----+---------+--------+---+------------+
|Title|Character|Dialogue| id|EpisodeTitle|
+-----+---------+--------+---+------------+
|    0|      220|       0|  0|           0|
+-----+---------+--------+---+------------+



In [ ]:
#As previously described, the remaining null character columns will be filled using the names found in dialogue (can be identified as text occuring
#before the ":" symbol). We first start by parsing the dualogue for when at the location of the ":" symbol and taking the first part of the parse
#since the speaker name comes before. Here we can also do a check to make sure that
#the length after parsesd text is >1 as this will easily indicate if a line of text was a dialogue or not. If the length is less than 1 then
#we replace the character name by "None" since it is not disloague. In fact most of the null character values did correspond to lines that are not
#dialogue but there were a small minority of lines that were null due to errors in the file so for the sake of being exact we are running the below
#line of code.
# We are carefull to only replace locations that are null with this new parsed speaker name and so only replace the character name
#if the value inside is null otherwise we keep it the same. We then filter out any remaining null values after this as these correspond to non dialogue
#lines
df_trans = df_trans.withColumn("New",when(size(split(col("Dialogue"),": "))>1,split(col("Dialogue"),": ")[0])\
                    .otherwise(None)).withColumn("New",regexp_replace(col("New"),'[^a-zA-Z ]',""))\
                    .withColumn("Character",when(col("Character").isNull(),col("New")).otherwise(col("Character")))\
                    .where(~col("Character").isNull()).select("Title","Character","Dialogue")    

#There are also two entries in the "Character" field that are not actual character and are instead plot from the story (not dialogue nor actions between square brackets)
#and so need to be removed. These two entries start with a leading white space and so we remove them by filtering out
#for character names that do not start with blank space.                  
df_trans = df_trans.filter(~df_trans.Character.startswith(" ")).withColumn("Character", lower(col("Character")))            

In [ ]:
#The transcripts dataframe is cleaned
df_trans.select([count(when(isnan(column) | col(column).isNull(), column)).alias(column) for column in df_trans.columns])\
                .show()

+-----+---------+--------+
|Title|Character|Dialogue|
+-----+---------+--------+
|    0|        0|       0|
+-----+---------+--------+



#  Exercise 7 - (2 points) Dialog number/Episode

<p align="justify">
<font size="3">
Provide the code that gives the number of dialogs for each episode.
</font>
</p>

In [ ]:
#We take a group by using the episode title and count the number of occurences. we sort by count and show

df_trans.select("Title").groupBy('Title').count().orderBy(col("count").desc()).show(185)

+--------------------+-----+
|               Title|count|
+--------------------+-----+
|06x17/18 - The De...|  597|
|07x11/12 - Classy...|  586|
|06x04/05 - Niagar...|  541|
|04x03/04 - Dunder...|  538|
|04x05/06 - Launch...|  537|
|   09x24/25 - Finale|  520|
|04x18/19 - Goodby...|  503|
|03x24/25 - The Jo...|  499|
|05x01/02 - Weight...|  496|
|  09x22/23 - A.A.R.M|  492|
|   09x16 - Moving On|  492|
|07x25/26 - Search...|  482|
|03x10/11 - A Beni...|  475|
|04x01/02 - Fun Ru...|  474|
|  03x08 - The Merger|  462|
|04x07/08 - Money ...|  445|
|05x14/15 - Stress...|  408|
|   07x05 - The Sting|  383|
|09x21 - Livin' th...|  376|
|05x07 - Customer ...|  361|
|02x22 - Casino Night|  356|
| 06x09 - Double Date|  355|
|  02x12 - The Injury|  352|
|  06x21 - Happy Hour|  351|
|03x19 - The Negot...|  348|
|         07x16 - PDA|  345|
|    01x06 - Hot Girl|  342|
|       06x06 - Mafia|  340|
| 08x16 - After Hours|  331|
|06x13 - Secret Santa|  329|
|      06x01 - Gossip|  328|
|07x22 - Goodb

#  Exercise 8 - (2 points) 

<p align="justify">
<font size="3">
Provide the code that returns the character that has the most number of dialogs (with the number of dialogs).



</font>
</p>

In [ ]:
#Since we can have different characters related ot the same dialogue as in part we we first need to parse the character field using the known
#parsing operators. Using the pos explode each of the parsed characters can be put on a seperate line. We do not need any other field since
#each character is linked ot a dialogue and so counting the characters occurences counts hte dialogues (We have already done a clean to rid
# of non dialogue lines before excercise 7). We then clean the character name to remove instances described beofre such as Michael [over the phone]
#Beth (Pams' mom) from the character name and keep only the name. We also remove any non letter elemtns formt he character name. These are done using
#the regexp_replace operation. A trim is performed to remove any trailing/leading white spaces before or after the character name. No need to lower case
#since this was done while cleaning the null values of the names.  We then group by the character name and count occurences. The counts are sorted
#and we show the highest count.

df_trans.select(posexplode(split(col("Character"),", |and |&")).alias("pos","Character")).select("Character")\
.select(posexplode(split(col("Character"),"\\/")).alias("pos","Character")).select("Character")\
.withColumn("Character",ltrim(rtrim(regexp_replace(regexp_replace("Character","[\(\[].*?[\)\]]",""),\
                    '[^a-zA-Z ]',""))))\
.select("Character").groupBy('Character').count().orderBy(col("count").desc()).show(1)

+---------+-----+
|Character|count|
+---------+-----+
|  michael|10697|
+---------+-----+
only showing top 1 row



#  Exercise 9 - (3 total points) 

<p align="justify">
<font size="3">




</font>
</p>

### 9.1 - (1 point) - Words/Serie Dataframe
Using a dataframe as input return the 10 most common words in the whole set of dialogs.

In [37]:
#Library to remove stopwords inside of a spark dataframe

from pyspark.ml.feature import StopWordsRemover

In [38]:
#The dialogue is first cleaned using the same logic as in part 1: we remove the speaker name by parsing at the ":" and take only what comes after,
#we remove any non dialogue things such as when a character is taking and doing an action (put in [] brackets) by removing what is contained isnide
#square brackets. we also remove any non letter items.We then parse the remaining dialogue by spaces to be able to performe a pos explode to 
#link each word to a seperate line. Since we want common words, we want to remove any words that are not stopwords. by creating a new column stop
#that contains either a word (if is not a stop ward) or "" (if it is a stop word). We then perform a where operation to only take the words in column
#stop that remain. A group by is performed and we count the number of words. Sort by count and show 10. 

df_temp = df_trans.withColumn("CleanDialogue",lower(regexp_replace(regexp_replace(split("Dialogue",":")[1],"[\(\[].*?[\)\]]",""),\
                    '[^a-zA-Z ]',""))).select(posexplode(split("CleanDialogue"," ")).alias("pos","words"))\
                    .select("words").withColumn("tokens", split("words", "\\s+"))

remover = StopWordsRemover(stopWords=stopwords_ENG, inputCol="tokens", outputCol="stop")

remover.transform(df_temp).select(col("words"), array_join("stop", " ").alias("stop"))\
.where((col("stop")!="")).groupBy("stop").count().orderBy(col("count").desc()).show(10)

+-----+-----+
| stop|count|
+-----+-----+
|   im| 4301|
| know| 3888|
|   oh| 3853|
| dont| 3594|
| like| 2932|
| yeah| 2770|
| okay| 2522|
| well| 2483|
|right| 2363|
|  get| 2306|
+-----+-----+
only showing top 10 rows



### 9.2 - (1 point) - Words/Character Dataframe
Using a dataframe as input return the 10 most common words for each character.

In [ ]:
#Since we need to group by the words by character, we first clean the dialogue as in 9.1. We then split dialogue with multiple characters
#as in in ex 8. We then remove stopwords as in 9.1. We now group by both character and word to get the count of each combination.
#The additional step here is to be able to find the 10 common words per character. So using the window operation we tell spark to partition windows 
#by character name and to order the count field generated by the previous group by. It will then assign an index (starting from 1) to each word per character
#depending on the desicending order of their count. This index is stored in a new column row. We now simply need to to filter from row number <=10 to have
#the top 10 words per character.

df_temp = df_trans.withColumn("CleanDialogue",lower(regexp_replace(regexp_replace(split("Dialogue",":")[1],"[\(\[].*?[\)\]]",""),\
                    '[^a-zA-Z ]',""))).select(posexplode(split(col("Character"),", |and |&")).alias("pos","Character"),"CleanDialogue").select("Character","CleanDialogue")\
                                      .select(posexplode(split(col("Character"),"\\/")).alias("pos","Character"),"CleanDialogue").select("Character","CleanDialogue")\
                                              .withColumn("Character",rtrim(ltrim(regexp_replace(regexp_replace("Character","[\(\[].*?[\)\]]",""),\
                    '[^a-zA-Z ]',""))))\
                    .select(posexplode(split("CleanDialogue"," ")).alias("pos","words"),"Character")\
                    .select("words","Character").withColumn("tokens", split("words", "\\s+"))

remover = StopWordsRemover(stopWords=stopwords_ENG, inputCol="tokens", outputCol="stop")

result = remover.transform(df_temp).select(col("Character"), array_join("stop", " ").alias("stop"))\
.where((col("stop")!="")).groupBy("Character","stop").count().orderBy(col("count").desc())

windowChar = Window.partitionBy("Character").orderBy(col("count").desc())

result.withColumn("row",row_number().over(windowChar))\
  .filter(col("row") <= 10).where(col("Character") !='')\
  .drop("row").show(1000, truncate = False)


+-----------------------+-------------+-----+
|Character              |stop         |count|
+-----------------------+-------------+-----+
|aaron rodgers          |youre        |3    |
|aaron rodgers          |lot          |2    |
|aaron rodgers          |good         |2    |
|aaron rodgers          |man          |2    |
|aaron rodgers          |heard        |1    |
|aaron rodgers          |play         |1    |
|aaron rodgers          |look         |1    |
|aaron rodgers          |enough       |1    |
|aaron rodgers          |today        |1    |
|aaron rodgers          |singers      |1    |
|abby                   |yeah         |2    |
|abby                   |e            |1    |
|abby                   |mrs          |1    |
|abby                   |frankweiler  |1    |
|abby                   |mixedup      |1    |
|abby                   |show         |1    |
|abby                   |aquarium     |1    |
|abby                   |sound        |1    |
|abby                   |files    

### 9.3 - (1 point) - Words/Episode Dataframe
Using a dataframe as input return the 10 episodes having the most number of words.

In [ ]:
#Dialogue is first cleaned as before. We then parse and explode the dialogue into words to each word is linked to the same information the dialogue
#was linked to before the parsing. We can now simply gorup by episode title ensuring that no '' were kept of the explode opetation by using the where
#clause to filter them out. We need to simply group by Episode title and count since each title is linked to a word as a results of the explode 
#operation. We sort by desscending order and take the top 10. 

df_trans.withColumn("CleanDialogue",lower(regexp_replace(regexp_replace(split("Dialogue",":")[1],"[\(\[].*?[\)\]]",""),\
                    '[^a-zA-Z ]',""))).select("Title",posexplode(split("CleanDialogue"," ")).alias("pos","words"))\
                    .select("Title","words").where(col("words") != "").groupBy("Title").count().orderBy(col("count").desc()).show(10, truncate = False)


+----------------------------------------------+-----+
|Title                                         |count|
+----------------------------------------------+-----+
|07x25/26 - Search Committee (Parts 1&2)       |6070 |
|07x11/12 - Classy Christmas (Parts 1&2)       |5935 |
|09x24/25 - Finale                             |5935 |
|06x17/18 - The Delivery (Parts 1&2)           |5885 |
|06x04/05 - Niagara (Parts 1&2)                |5552 |
|04x03/04 - Dunder Mifflin Infinity (Parts 1&2)|5448 |
|04x05/06 - Launch Party (Parts 1&2)           |5361 |
|05x01/02 - Weight Loss (Parts 1&2)            |5212 |
|09x16 - Moving On                             |5212 |
|04x01/02 - Fun Run (Parts 1&2)                |5201 |
+----------------------------------------------+-----+
only showing top 10 rows



# 2 - Bonus 


<p align="justify">
<font size="3">
Comment the complexity and the differences between Exercise 3 and Exercise 9.

Which approach do you find better? In which part of the analysis? Why?

</font>
</p>




The results between the two excercises were conceptually the same (ie the order of the output is the same). There are negligible differences in the word counts which oculd be attrbuted to small variations in the methods used to clean the files. I found that in excercise 9 it was easier
to find mistakes in the code as well as to be able to spot things that I may have forgot to consider (such as an extra parsing seperator). Also, as long as you know the correct functionsm, it is much easier to carry out operations in Excercise 9. What was especially interesting is the runtime
of operations between Ex 3 and 9. It seems as though all the taks in Part 2 in general run in split seconds compared to Part 1 that can take a few to many seconds. This is especially true for Reduce operations. It seems the method of reduction in rdd oeprations take much longer than in a spark dataframe and using a group by count method. One operation that was noticeably running slower in the Spark Dataframes compared to others is finding null values. It seems that filtering operations in rdd are quicker. Overall Part 1 is better, to understand the Python logic that needs to be carried out but Part 2 (after aving proficiency with the operations) is easier to use and to be thorough with the data. Loading the data was of course quicker in Part 1 however this is a conseuqnece of the rdd being lazy in not carrying out any operation until an action being called which resulted in the slowing of each task. This can still be reassuring since most of the time we only need to create summaries from big amounts of data so what needs to be outputed is significantly smaller than the other data. THe spark dataframes are however visually better if we want to get an idea of how the data looks (this is esppecially true because to take even 1 entr from the original files in rdd takes a long time). Despite this seemingly biased opinion against Part 1, I actually enjoyed working on it more. 
